In [2]:
from __future__ import unicode_literals, print_function, division
from io import open
import glob
import os

In [3]:
def findFiles(path):
    return glob.glob(path)

In [4]:
print(findFiles('data/data/names/*.txt'))

['data/data/names\\Arabic.txt', 'data/data/names\\Chinese.txt', 'data/data/names\\Czech.txt', 'data/data/names\\Dutch.txt', 'data/data/names\\English.txt', 'data/data/names\\French.txt', 'data/data/names\\German.txt', 'data/data/names\\Greek.txt', 'data/data/names\\Irish.txt', 'data/data/names\\Italian.txt', 'data/data/names\\Japanese.txt', 'data/data/names\\Korean.txt', 'data/data/names\\Polish.txt', 'data/data/names\\Portuguese.txt', 'data/data/names\\Russian.txt', 'data/data/names\\Scottish.txt', 'data/data/names\\Spanish.txt', 'data/data/names\\Vietnamese.txt']


In [5]:
import unicodedata
import string

In [6]:
%load_ext autoreload
%autoreload 2

In [7]:
all_letters = string.ascii_letters + " .,;'"
n_letters = len(all_letters)

In [8]:
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
        and c in all_letters
    )
print(unicodeToAscii('Ślusàrski'))

Slusarski


In [9]:
category_lines = {}
all_categories = []

In [10]:
def readLines(filename):
    lines = open(filename, encoding='utf-8').read().strip().split('\n')
    return [unicodeToAscii(line) for line in lines]

In [11]:
for filename in findFiles('data/data/names/*.txt'):
    category = os.path.splitext(os.path.basename(filename))[0]
    all_categories.append(category)
    lines = readLines(filename)
    category_lines[category] = lines
    
n_categories = len(category_lines)

In [12]:
print(category_lines['Italian'][:5])

['Abandonato', 'Abatangelo', 'Abatantuono', 'Abate', 'Abategiovanni']


In [13]:
from process_text_to_tensor import *

In [14]:
print(letterToTensor(all_letters, 'J', n_letters))

tensor([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]])


In [15]:
print(lineToTensor(all_letters, 'Jones', n_letters).size())

torch.Size([5, 1, 57])


In [16]:
from rnn import RNN

In [17]:
n_hidden = 128
rnn = RNN(n_letters, n_hidden, n_categories)

In [18]:
import torch

In [19]:
input = letterToTensor(all_letters, 'Albert', n_letters)
hidden = torch.zeros(1, n_hidden)

output, next_hidden = rnn(input, hidden)
print(output)

tensor([[-2.9151, -2.8560, -2.8822, -2.8544, -2.8597, -2.8455, -2.8369,
         -2.8721, -2.7714, -2.9035, -2.9219, -2.9266, -2.9679, -2.9388,
         -2.9232, -2.9778, -2.9168, -2.8791]])


In [20]:
def categoryFromOutput(output):
    top_n, top_i = output.topk(1)
    category_i = top_i[0][0]
    return all_categories[category_i], category_i

In [21]:
print(categoryFromOutput(output))

('Irish', tensor(8))


In [22]:
import random

def randomChoice(l):
    return l[random.randint(0, len(l) - 1)]

def randomTrainingExample():
    category = randomChoice(all_categories)
    line = randomChoice(category_lines[category])
    category_tensor = torch.tensor([all_categories.index(category)], dtype=torch.long)
    line_tensor = lineToTensor(all_letters, line, n_letters)
    return category, line, category_tensor, line_tensor
    

In [23]:
for i in range(10):
    category, line, category_tensor, line_tensor = randomTrainingExample()
    print(category, line)

Spanish Perez
German Tiedeman
Arabic Ghannam
Scottish Williamson
Dutch Can
Chinese Qing
Greek Metrofanis
Irish Donnchadh
Irish O'Doherty
Irish Ruadh


In [24]:
import torch.nn as nn
learning_rate = 0.001
criterion = nn.NLLLoss()

def train(category_tensor, line_tensor):
    hidden = rnn.initHidden()
    rnn.zero_grad()
    
    for i in range(line_tensor.size()[0]):
        output, hidden = rnn(line_tensor[i], hidden)
        
    loss = criterion(output, category_tensor)
    loss.backward()
    
    for p in rnn.parameters():
        p.data.add_(-learning_rate, p.grad.data)
        
    return output, loss.item()

In [25]:
import time
import math
n_iters = 100000
print_every = 5000
plot_every = 1000

current_loss = 0
all_losses = []

def timeSince(since):
    now = time.time()
    s = now - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

start = time.time()

for iter in range(1, n_iters + 1):
    cateogry, line, category_tensor, line_tensor = randomTrainingExample()
    output, loss = train(category_tensor, line_tensor)
    current_loss += loss
    
    if iter % print_every == 0:
        guess, guess_i = categoryFromOutput(output)
        correct = '✓' if guess == category else '✗ (%s)' % category
        print('%d %d%% (%s) %.4f %s / %s %s' % (iter, iter / n_iters * 100, timeSince(start), loss, line, guess, correct))

    if iter % plot_every == 0:
        all_losses.append(current_loss / plot_every)
        current_loss = 0

5000 5% (0m 7s) 2.9678 Amari / Japanese ✗ (Irish)
10000 10% (0m 14s) 2.8831 Paul / Dutch ✗ (Irish)
15000 15% (0m 21s) 2.6660 Guerra / Japanese ✗ (Irish)
20000 20% (0m 29s) 2.2745 Garofalo / Italian ✗ (Irish)
25000 25% (0m 37s) 2.5491 Lauwers / French ✗ (Irish)
30000 30% (0m 44s) 2.5236 Bancroft / Italian ✗ (Irish)
35000 35% (0m 51s) 1.0368 Petrakis / Greek ✗ (Irish)
40000 40% (0m 58s) 2.2969 Paterson / Russian ✗ (Irish)
45000 45% (1m 6s) 1.3250 Assaf / Arabic ✗ (Irish)
50000 50% (1m 13s) 2.6787 Rendon / Scottish ✗ (Irish)
55000 55% (1m 20s) 2.8744 Hoang / Arabic ✗ (Irish)
60000 60% (1m 27s) 3.2916 Macdermott / French ✗ (Irish)
65000 65% (1m 33s) 1.5278 Geiger / German ✗ (Irish)
70000 70% (1m 40s) 1.3157 Sabbag / Arabic ✗ (Irish)
75000 75% (1m 47s) 1.6985 Rypka / Polish ✗ (Irish)
80000 80% (1m 55s) 1.4762 Klimek / Polish ✗ (Irish)
85000 85% (2m 4s) 2.0326 Reid / Scottish ✗ (Irish)
90000 90% (2m 11s) 1.1250 Meeuwsen / Dutch ✗ (Irish)
95000 95% (2m 19s) 2.2892 Medeiros / Greek ✗ (Irish)
1